In [ ]:
import IPython
from dotenv import load_dotenv
import os 
import vertexai
from IPython.display import Markdown, Video, display 
import pandas as pd
from io import StringIO
import json

from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    Tool,
    FunctionDeclaration,
)

In [ ]:
load_dotenv()

In [ ]:
project_id = os.getenv('PROJECT_ID')
location = "us-west1"

vertexai.init(project=project_id, location=location)

In [ ]:
model = GenerativeModel("gemini-1.5-pro-001")

generation_config = GenerationConfig(temperature=1, top_p=0.95)

safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

In [ ]:
instructions = """  
Please return JSON of the vernacular name of the fish and coral species, the scientific name, and the best timestamp of the frame identified from the video. Use this
example following the schema:

    {"vernacular name": str, "scientific name": str, "timestamp": str}

    All fields are required.

               """

In [ ]:
def get_url_from_gcs(gcs_uri: str) -> str:
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )

In [ ]:
video_uri = 'gs://fish-dataset-test/sharktest2.mp4'



contents = [
    Part.from_uri(
        uri=video_uri,
        mime_type="video/mp4",
    ),
    instructions
]

display(Video(get_url_from_gcs(video_uri)))

In [ ]:
response = model.generate_content(contents, generation_config=generation_config)



In [ ]:
print(response.text)

In [ ]:
data = response.text.replace('*', '').replace("\n", "").replace('json', "").replace("`", "")

In [ ]:
data = json.loads(data)

In [ ]:
data

In [ ]:
df = pd.DataFrame(data, index=[0])

In [ ]:
df

In [ ]:
for key in data:
    print(key.get('vernacular name'))

In [ ]:
import dropbox


    # Create a Dropbox client
    dbx = dropbox.Dropbox(dropbox_token)

 

    try:
        link = dbx.files_get_temporary_link(file_path)
        return link.link
    except dropbox.exceptions.ApiError as e:
        print(f"Error accessing file: {e}")

var = upload_to_dropbox('/FRAMES/A_Green_Turtle.jpg')
print(f'{var}')

In [ ]:
file_path = '/FRAMES/thresher_shark.jpg' # Add leading slash and check case

try:
    link = dbx.files_get_temporary_link(file_path)
    print(link)
except dropbox.exceptions.ApiError as e:
    print(f"Error accessing file: {e}")